<p>
<img src="https://alanattableau.files.wordpress.com/2014/12/cropped-databender4.png" align=center>
<h1 align=center>The Last Databenders</h1>
<h4 align=center>Sponsored By Gutenberg</h4>
<img src="http://www.gutenberg.org/pics/logo-144x144.png" align=center>
</p>

In [ ]:
import urllib2, string, json  # the lib that handles the url stuff
from bs4 import BeautifulSoup
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover
from pyspark.sql import Row

HEADER_MARKERS = [

      '* START OF THE PROJECT GUTENBERG',
      '* START OF THIS PROJECT GUTENBERG',
      'This etext was prepared by',
      'E-text prepared by',
      'Produced by',
      'Distributed Proofreading Team',
      '*END THE SMALL PRINT',
      '***START OF THE PROJECT GUTENBERG',
      'This etext was produced by',
      '* START OF THE COPYRIGHTED',
      'The Project Gutenberg',
      'http://gutenberg.spiegel.de/ erreichbar.',
      'Project Runeberg publishes',
      'Beginning of this Project Gutenberg',
      'Project Gutenberg Online Distributed',
      'Gutenberg Online Distributed',
      'the Project Gutenberg Online Distributed',
      'Project Gutenberg TEI',
      'This eBook was prepared by',
      'http://gutenberg2000.de erreichbar.',
      'This Etext was prepared by',
      'Gutenberg Distributed Proofreaders',
      'the Project Gutenberg Online Distributed Proofreading Team',
      '**The Project Gutenberg',
      '*SMALL PRINT!',
      'More information about this book is at the top of this file.',
      'tells you about restrictions in how the file may be used.',
      'of the etext through OCR.',
      '**These eBooks Were Prepared By Thousands of Volunteers!**',
      'SERVICE THAT CHARGES FOR DOWNLOAD',
      'We need your donations more than ever!',
      ' * START OF THIS PROJECT GUTENBERG',
      '**     SMALL PRINT!'
    
]
FOOTER_MARKERS = [
    '* END OF THE PROJECT GUTENBERG',
    '* END OF THIS PROJECT GUTENBERG',
    '***END OF THE PROJECT GUTENBERG',
    'End of the Project Gutenberg',
    'End of The Project Gutenberg',
    'by Project Gutenberg',
    'End of Project Gutenberg',
    'End of this Project Gutenberg',
    '        ***END OF THE PROJECT GUTENBERG',
    '* END OF THE COPYRIGHTED',
    'End of this is COPYRIGHTED',
    '*This is a COPYRIGHTED Project Gutenberg Etext, Details Above*',
    'More information about this book is at the top of this file.',
    'We need your donations more than ever!',
    '<<THIS ELECTRONIC VERSION OF',
    'END OF PROJECT GUTENBERG',
    ' End of the Project Gutenberg',
    ' * END OF THIS PROJECT GUTENBERG',
    "THE END"  
]

stopworddd = ['a', "a's", 'able', 'about', 'above', 'according',"project","gutenberg","ebook",'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'b', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'c', "c'mon", "c's", 'came', 'can', "can't", 'cannot', 'cant', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly',"the", 'co', 'com', 'come', 'comes', 'concerning', 'consequently', 'consider', 'considering', 'contain', 'containing', 'contains', 'corresponding', 'could', "couldn't", 'course', 'currently', 'd', 'definitely', 'described', 'despite', 'did', "didn't", 'different', 'do', 'does', "doesn't", 'doing', "don't", 'done', 'down', 'downwards', 'during', 'e', 'each', 'edu', 'eg', 'eight', 'either', 'else', 'elsewhere', 'enough', 'entirely', 'especially', 'et', 'etc', 'even', 'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'ex', 'exactly', 'example', 'except', 'f', 'far', 'few', 'fifth', 'first', 'five', 'followed', 'following', 'follows', 'for', 'former', 'formerly', 'forth', 'four', 'from', 'further', 'furthermore', 'g', 'get', 'gets', 'getting', 'given', 'gives', 'go', 'goes', 'going',"things","electronic work","electronic","work",'gone', 'got', 'gotten', 'greetings', 'h', 'had', "hadn't", 'happens', 'hardly', 'has', "hasn't", 'have', "haven't", 'having', 'he', "he's", 'hello', 'help', 'hence', 'her', 'here', "here's", 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'hi', 'him', 'himself', 'his', 'hither', 'hopefully', 'how', 'howbeit', 'however', 'i', "i'd", "i'll", "i'm", "i've", 'ie', 'if', 'ignored', 'immediate', 'in', 'inasmuch', 'inc', 'indeed', 'indicate', 'indicated', 'indicates', 'inner', 'insofar', 'instead', 'into', 'inward', 'is', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', 'j', 'just', 'k', 'keep', 'keeps', 'kept', 'know', 'knows', 'known', 'l', 'last', 'lately', 'later', 'latter', 'latterly', 'least', 'less', 'lest', 'let', "let's", 'like', 'liked', 'likely', 'little', 'look', 'looking', 'looks', 'ltd', 'm', 'mainly', 'many', 'may', 'maybe', 'me', 'mean', 'meanwhile', 'merely', 'might', 'more', 'moreover', 'most', 'mostly', 'much', 'must', 'my', 'myself', 'n', 'name', 'namely', 'nd', 'near', 'nearly', 'necessary', 'need', 'needs', 'neither', 'never', 'nevertheless', 'new', 'next', 'nine', 'no', 'nobody', 'non', 'none', 'noone', 'nor', 'normally', 'not', 'nothing', 'novel', 'now', 'nowhere', 'o', 'obviously', 'of', 'off', 'often', 'oh', 'ok', 'okay', 'old', 'on', 'once', 'one', 'ones', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'ought', 'our', 'ours', 'ourselves', 'out', 'outside', 'over', 'overall', 'own', 'p', 'particular', 'particularly', 'per', 'perhaps', 'placed', 'please', 'plus', 'possible', 'presumably', 'probably', 'provides', 'q', 'que', 'quite', 'qv', 'r', 'rather', 'rd', 're', 'really', 'reasonably', 'regarding', 'regardless', 'regards', 'relatively', 'respectively', 'right', 's', 'said', 'same', 'saw', 'say', 'saying', 'says', 'second', 'secondly', 'see', 'seeing', 'seem', 'seemed', 'seeming', 'seems', 'seen', 'self', 'selves', 'sensible', 'sent', 'serious', 'seriously', 'seven', 'several', 'shall', 'she', 'should', "shouldn't", 'since', 'six', 'so', 'some', 'somebody', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhat', 'somewhere', 'soon', 'sorry', 'specified', 'specify', 'specifying', 'still', 'sub', 'such', 'sup', 'sure', 't', "t's", 'take', 'taken', 'tell', 'tends', 'th', 'than', 'thank', 'thanks', 'thanx', 'that', "that's", 'thats', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there', "there's", 'thereafter', 'thereby',"place","moment", 'therefore', 'therein', 'theres', 'thereupon', 'these', 'they', "they'd", "they'll", "they're", "they've", 'think', 'third', 'this', 'thorough', 'thoroughly', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'took', 'toward', 'towards', 'tried', 'tries', 'truly', 'try', 'trying', 'twice', 'two', 'u', 'un', 'under', 'unfortunately', 'unless', 'unlikely', 'until', 'unto', 'up', 'upon', 'us', 'use', 'used', 'useful', 'uses', 'using', 'usually', 'uucp', 'v', 'value', 'various', 'very', 'via', 'viz', 'vs', 'w', 'want', 'wants', 'was', "wasn't", 'way', 'we', "we'd", "we'll", "we're", "we've", 'welcome', 'well', 'went', 'were', "weren't", 'what', "what's", 'whatever', 'when', 'whence', 'whenever', 'where', "where's", 'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', "who's", 'whoever', 'whole', 'whom', 'whose', 'why', 'will', 'willing', 'wish', 'with', 'within', 'without', "won't", 'wonder', 'would', 'would', "wouldn't", 'x', 'y', 'yes', 'yet', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', 'z', 'zero',"made","time","thing","man","hand","men","word","thought","day","electronic work","found"]
adjectives = ['able', 'bad', 'best',"back", 'better', 'big', 'black', 'certain', 'clear', 'early', 'easy', 'free', 'full', 'good', 'great', 'hard', 'high', 'important', 'large', 'late', 'little', 'local', 'long', 'low', 'major', 'new', 'old', 'only', 'other', 'political', 'possible', 'public', 'real', 'recent', 'right', 'small', 'social', 'special', 'strong', 'sure', 'true', 'white', 'whole', 'young']
common_verbs = ['say', 'make', 'go', 'take', 'come', 'see', 'know', 'get', 'give', 'find', 'think', 'tell', 'become', 'show', 'leave', 'feel', 'put', 'bring', 'begin', 'keep', 'hold', 'write', 'stand', 'hear', 'let', 'mean', 'set', 'meet', 'run', 'pay', 'sit', 'speak', 'lie', 'lead', 'read', 'grow', 'lose', 'fall', 'send', 'build', 'draw', 'break', 'spend', 'cut', 'rise', 'drive', 'buy', 'wear', 'choose', 'said', 'made', 'went', 'took', 'came', 'saw', 'knew', 'got', 'gavefound', 'thought', 'told', 'became', 'showed', 'left', 'felt', 'put', 'brought', 'began', 'kept', 'held', 'wrote', 'stood', 'heard', 'let', 'meant', 'set', 'met', 'ran', 'paid', 'sat', 'spoke', 'lay', 'led', 'read', 'grew', 'lost', 'fell', 'sent', 'built', 'drew', 'broke', 'spent', 'cut', 'rose', 'drove', 'bought', 'wore', 'chose', 'gone', 'taken', 'seen', 'known', 'given', 'shown', 'written', 'spoken', 'lain', 'grown', 'fallen', 'drawn', 'broken', 'risen', 'driven', 'worn', 'chosen', 'wanted', 'used', 'worked', 'called', 'tried', 'asked', 'needed', 'seemed', 'helped', 'played', 'moved', 'lived', 'believed', 'happened', 'included', 'continued', 'changed', 'watched', 'followed', 'stopped', 'created', 'opened', 'walked', 'offered', 'remembered', 'appeared', 'served', 'died', 'stayed', 'reached', 'killed', 'raised', 'passed', 'decided', 'returned', 'explained', 'hoped', 'carried', 'received', 'agreed', 'covered', 'caused', 'closed', 'increased', 'reduced', 'noted', 'entered', 'shared', 'saved', 'protected', 'occurred', 'accepted', 'determined', 'prepared', 'argued', 'recognized', 'indicated', 'arrived', 'answered', 'compared', 'acted', 'studied', 'removed', 'sounded', 'formed', 'visited', 'avoided', 'imagined', 'finished', 'responded', 'maintained', 'contained', 'applied', 'treated', 'affected', 'worried', 'mentioned', 'improved', 'signed', 'existed', 'noticed', 'travelled', 'prevented', 'admitted', 'suffered', 'published', 'counted', 'achieved', 'announced', 'touched', 'attended', 'defined', 'introduced']
final_stopwords = stopworddd + adjectives + common_verbs
categories = ["Adventure", "Classic", "History", "Horror", "Science-fiction", "Romance", "Fantasy", "Children"]

def ParseBook(data):
    is_header = False
    is_footer = False
    for line in data:
        for footer in FOOTER_MARKERS:
            if footer in line.strip():
                is_footer = True
                break
        if is_footer:
            break
        if is_header:
            for i in line.split():
                word = i.strip().lower().translate(None, string.punctuation)
                tx.write(word+" ")

        for header in HEADER_MARKERS:
            if header in line.strip():
                is_header = True
                break
                
def FillDict(index, cat):
    tx.write(cat+"\t")
    try:
        data = urllib2.urlopen("http://www.gutenberg.org/files/"+str(index)+"/"+str(index)+"-0.txt") # it's a file like object and works just like a file
        ParseBook(data)
    except urllib2.HTTPError:
        try:
            data = urllib2.urlopen("http://www.gutenberg.org/files/"+str(index)+"/"+str(index)+".txt") # it's a file like object and works just like a file
            ParseBook(data)
        except:
            pass
    tx.write("\n")
    
def FindBookIndex(category):
    categorise = [category]
    for cat in categorise:
        book_index = 0
        print cat
        while book_index < 200:
            try:
                html_doc = urllib2.urlopen("http://www.gutenberg.org/ebooks/search/?start_index="+str(book_index)+"&query="+cat+"+l.english").read()
                soup = BeautifulSoup(html_doc, 'html.parser')
                lis = soup.find_all("li",  { "class" : "booklink" })
                for i in range(len(lis)):
                    index = lis[i].a['href'].split("/")[2]
                    FillDict(index, cat)
                book_index += 26
            except urllib2.HTTPError:
                break

for i in categories:
    tx = open("%s_b.txt" % (i), "w")
    FindBookIndex(i)
    tx.close()
    print "%s DONE" % (i)

Adventure
Adventure DONE
Classic
Classic DONE
History
History DONE
Horror
Horror DONE
Science-fiction
Science-fiction DONE
Romance
Romance DONE
Fantasy
Fantasy DONE
Children
Children DONE
